This file has most of the code done in python for this project. For the sake of documentation, there was an attempt to delete the minimum amount of code possible, so, consequentialy, some sections have poor results.The structure of this file follows the "chronological" order of this work, which results in the code and commentaries feeling incoherent at times and the overall objective hard to follow, however, it also shows the evolution of the work and some of the story behind this project.



The first major part of this project is to study, and analize the potential of the LSM6DSOX accelerometer. This accelerometer has many interesting features and usefull functions that i will not refer here, however, it was important to study the MLC capabilities of this accelerometer since it can provide a great low power solution to the classification problem. This study was done in 3 steps:

1-Analize raw data gathered from the accelerometer.

2-Analize the features utilized in the internal MLC.

3-Analize results gathered from the internal MLC.


This first snipit of code has the necessary includes and some function definitions for later use. Most of the code in this file is independent,however, most sections depend on this one, therefore it is important to always run this code before executing any other section.

P2P stands for peak to peak. This function does the subtraction between the maximum and minimum of the signal.

Var stand for variance. This function does the following equation: $ (\frac{\sum_{k=0}^{WL-1} I_{k}^{2}}{WL}) - (\frac{\sum_{k=0}^{WL-1} I_{k}}{WL})^{2} $

Energy is self explanatory and is applies the following equation: $ \sum_{k=0}^{WL-1} I_{k}^{2} $

All of these functions were taken from the official application note:
https://www.st.com/content/ccc/resource/technical/document/application_note/group1/5f/d8/0a/fe/04/f0/4c/b8/DM00563460/files/DM00563460.pdf/jcr:content/translations/en.DM00563460.pdf

In [2]:
import serial as serial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
import scipy.signal
import plotly.express as ex
from numpy.fft import fft,fftfreq
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import librosa
from sklearn.cluster import KMeans

def p2p(data,window_size):
    result=[]
    i=0
    data_window=[]
    for x in data:
        i+=1
        data_window.append(x)
        if i>=window_size:
            i=0
            result.append(max(data_window)-min(data_window))
            data_window=[]
    return result

def var(data,window_size):
    result=[]
    i=0
    data_window=[]
    data_window_2=[]
    for x in data:
        i+=1
        data_window.append(x)
        data_window_2.append(x*x)
        if i>=window_size:
            i=0
            second=(sum(data_window)/window_size)
            result.append((sum(data_window_2)/window_size)-(second*second))
            data_window=[]
            data_window_2=[]
    return result

def energy(data,window_size):
    result=[]
    i=0
    data_window=[]
    for x in data:
        i+=1
        data_window.append(x*x)
        if i>=window_size:
            i=0
            result.append(sum(data_window))
            data_window=[]
    return result

def process_raw(raw_lines):
    result=[]
    med_x=0
    med_y=0
    med_z=0
    for line in raw_lines:
        str_list=list(line.strip().split(" "))
        try:
            float_list=[float(x) for x in str_list]
            med_x+=float_list[0]
            med_y+=float_list[1]
            med_z+=float_list[2]
            result.append(float_list)
        except:
            continue
    med_x=med_x/len(result)
    med_y=med_y/len(result)
    med_z=med_z/len(result)
    return result,med_x,med_y,med_z


def process_features(raw_lines,n_of_samples,only_FFT):
    peak_amp_or_val=0
    mean_flag=0
    sample_idx=0
    Dataset=np.empty((3),dtype=object)
    peaks=np.empty((3),dtype=object)
    peaks_amp=np.empty((3),dtype=object)
    mean_std=np.empty((3),dtype=object)
    for line in raw_lines:
        if(line=="y:  \n"):
            sample_idx=0
            line_state=1
            idx=0
        elif(line=="z:  \n"):
            sample_idx=0
            line_state=2
            idx=0
        str_list=list(line.strip().split(" "))
        if(sample_idx!=n_of_samples):
            try:
                float_list=[float(x) for x in str_list]
                if(idx==0):
                    Dataset[line_state]=[]
                    peaks[line_state]=[]
                    peaks_amp[line_state]=[]
                    mean_std[line_state]=[]
                    idx=1
                if(fft_or_peak==0 and only_FFT==1):
                    Dataset[line_state].append(float_list)
                    fft_or_peak=1
                else:
                    if(peak_amp_or_val==0):
                        peaks[line_state].append(float_list)
                        peak_amp_or_val=1
                    else:
                        if mean_flag==0:
                            peaks_amp[line_state].append(float_list)
                            mean_flag=1
                        else:
                            mean_std[line_state].append(float_list)
                            fft_or_peak=0
                            peak_amp_or_val=0
                            mean_flag=0   
                            sample_idx+=1
            except:
                continue
    return Dataset,peaks,peaks_amp,mean_std

def build_dataset(FFTs,peak_vals,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std):
    X_train=[]
    X_test=[]
    y_train=[]
    y_test=[]
    n_of_samples=len(FFTs[0][0])
    train_samples=int(n_of_samples*(1-dataset_ratio))
    if peaks_or_fft==1:
        for class_n,all_samples_in_class in enumerate(peak_vals):
            for sample_n in range(len(all_samples_in_class[0])):
                sample=[]
                for axis in range(3):
                    axis_sample=[]
                    axis_amp=[]
                    for n_of_peaks in range(len(all_samples_in_class[0][0])):
                        axis_sample.append(all_samples_in_class[axis][sample_n][n_of_peaks])
                        axis_amp.append(peak_amp[class_n][axis][sample_n][n_of_peaks])
                    max_peak=axis_amp[0]
                    if sort==1:
                        sort_minus_one=0
                        c = []
                        for i in range(len(axis_sample)):
                            if axis_sample[i]==-1:
                                sort_minus_one+=1
                            else:
                                c.append((axis_sample[i], axis_amp[i]))
                        r = sorted(c)
                        for i in range(len(r)):
                            axis_sample[i], axis_amp[i] = r[i]
                        for i in range(sort_minus_one):
                            axis_sample.append(-1)
                            axis_amp.append(0)
                    for peak in range(n_of_peaks_to_use):
                        sample.append(axis_sample[peak])
                        if use_of_amps==1:
                            sample.append(axis_amp[peak])
                    if add_energy==1:
                        sum_of_fft=0
                        for val in FFTs[class_n][axis][sample_n]:
                            sum_of_fft+=val
                        sample.append(sum_of_fft/len(FFTs[class_n][axis][sample_n]))
                    if add_mean==1:
                        sample.append(mean_std[class_n][axis][sample_n][0])
                    if add_std==1:
                        sample.append(mean_std[class_n][axis][sample_n][1])
                if(sample_n<train_samples):
                    X_train.append(sample)
                    y_train.append(class_n)
                else:
                    X_test.append(sample)
                    y_test.append(class_n)
    elif peaks_or_fft==0:
        for class_n,all_samples_in_class in  enumerate(FFTs):
            for sample_n in range(len(all_samples_in_class[1])):
                sample=[]
                for axis in  range(3):
                    aux=all_samples_in_class[axis][sample_n][:len(all_samples_in_class[axis][sample_n])]
                    sample.extend(aux)
                if(sample_n<train_samples):
                    X_train.append(sample)
                    y_train.append(class_n)
                else:
                    X_test.append(sample)
                    y_test.append(class_n)
    return X_train,y_train,X_test,y_test
def port_kmeans_to_C(kmeans,file_name,X_train,quantize):
    #first it will determine the maximum distance of each inner cluster, that is, the distance of the furtherst training sample of each cluster
    centroid_list=kmeans.cluster_centers_
    max_dist=[0]*len(centroid_list)
    for sample_idx,sample in enumerate(X_train):
        min_dist=0
        min_aux=1
        assigned_centroid=0
        for centroid_idx,centroid in enumerate(centroid_list):
            distance=0
            for feature in range(len(sample)):
                distance+=np.sqrt((sample[feature]-centroid[feature])**2)
            if min_aux==1:
                min_dist=distance
                assigned_centroid=centroid_idx
                min_aux=0
            elif distance<min_dist:
                min_dist=distance
                assigned_centroid=centroid_idx
        if min_dist>max_dist[assigned_centroid]:
            max_dist[assigned_centroid]=min_dist
    #this will now make the code, in which the template is constant but the values are need to be filled
    max_dist_str=""
    cluster_centers=""
    #quantization is simply turning every value into int, the logic is the same
    if quantize==1:
        #first the max_distances
        max_dist_str="int max_dist["+str(len(max_dist))+"]={"+str(int(max_dist[0]))
        for val_idx,val in enumerate(max_dist):
            if val_idx==0:
                continue
            max_dist_str+=","+str(int(val))
        max_dist_str+="};"
        #now the cluster centers
        cluster_centers="int clusters["+str(len(max_dist))+"]["+str(len(X_train[0]))+"]={"
        for cluster_idx,cluster in enumerate(kmeans.cluster_centers_):
            cluster_centers+="{"+str(int(cluster[0]))
            for val_idx,val in enumerate(cluster):
                if val_idx==0:
                    continue
                else:
                    cluster_centers+=","+str(int(val))
            if cluster_idx!=len(kmeans.cluster_centers_)-1:
                cluster_centers+="},"
        cluster_centers+="}};"
    else:
        max_dist_str="float max_dist["+str(len(max_dist))+"]={"+str(max_dist[0])
        for val in max_dist:
            if val==max_dist[0]:
                continue
            max_dist_str+=","+str(val)
        max_dist_str+="};"
        cluster_centers="float clusters["+str(len(max_dist))+"]["+str(len(X_train[0]))+"]={"
        for cluster_idx,cluster in enumerate(kmeans.cluster_centers_):
            cluster_centers+="{"+str(cluster[0])
            for val_idx,val in enumerate(cluster):
                if val_idx==0:
                    continue
                else:
                    cluster_centers+=","+str(val)
            if cluster_idx!=len(kmeans.cluster_centers_)-1:
                cluster_centers+="},"
        cluster_centers+="}};"
    code_template='''\
    #include <iostream>
    #include <cmath>
    using namespace std;

    class KMeans{
        public:
            %s
            %s
            int predict(float* sample);
    };
    int KMeans::predict(float * sample){
        float min_distance=0;
        bool min_aux=1;
        int res=0;
        for(int i=0;i<int(%s);i++){
            float distance=0;
            for(int j=0;j<int(%s);j++){
                distance=distance+sqrt((sample[j]-KMeans::clusters[i][j])*(sample[j]-KMeans::clusters[i][j]));
            }
            cout<<distance<<endl;
            if(min_aux==1){
                min_distance=distance;
                min_aux=0;
                res=i;
            }
            else{
                if(min_distance>distance){
                    min_distance=distance;
                    res=i;
                }
            }
        }
        if(min_distance>max_dist[res]){
            return -1;
        }
        return res;
    }
    '''
    c_code=code_template % (max_dist_str,cluster_centers,len(max_dist),len(X_train[0]))
    with open(file_name+".h",'w') as file:
        file.write(c_code)


def port_scaller(scaller,file_name,quantize):
    mean_vals=scaller.mean_
    var_vals=scaller.var_
    var_str=""
    mean_str=""
    if quantize==1:
        mean_str="int means["+str(len(mean_vals))+"]={"+str(int(mean_vals[0]))
        var_str="int vars["+str(len(var_vals))+"]={"+str(int(math.sqrt(var_vals[0])))
        for val_idx in range(len(mean_vals)):
            if val_idx!=0:
                mean_str+=","+str(int(mean_vals[val_idx]))
                var_str+=","+str(int(math.sqrt(var_vals[val_idx])))
        mean_str+="};"
        var_str+="};"
    else:
        mean_str="int means["+str(len(mean_vals))+"]={"+str(mean_vals[0])
        var_str="int vars["+str(len(var_vals))+"]={"+str(math.sqrt(var_vals[0]))
        for val_idx in range(len(mean_vals)):
            if val_idx!=0:
                mean_str+=","+str(mean_vals[val_idx])
                var_str+=","+str(math.sqrt(var_vals[val_idx]))
        mean_str+="};"
        var_str+="};"
    code_template='''\
    #include <iostream>

    class Scaller{
        public:
            %s
            %s
            void transform(float* sample,float* dest);      
    };

    void Scaller::transform(float* sample,float* dest){
        float new_sample[21]={0};
        for(int i=0;i<21;i++){
            new_sample[i]=(sample[i]-means[i])/vars[i];
        }
        memcpy(dest, new_sample, sizeof(float) * 21);

    }
    '''
    c_code=code_template % (mean_str,var_str)
    with open(file_name+".h",'w') as file:
        file.write(c_code)

    
    
init_dir=os.getcwd()


The following code turns the seperate axis into a joint axis. There are some inputs made by the user to either filter throught a high pass filter ot take of the averages calculated in the previous snipit. It is important to note that, if the user utilized the HP filter, it will print the coeficients utilized. These coeficients can be utilized in the MLC. The result is ploted.

The MLC selectable features is calculated next and its respective results are shown.

In [ ]:
os.chdir(init_dir)
os.chdir(r'..\Data\actual_new_motor')
file_idle=open("state20.txt",'r')
file_shake=open("state21.txt",'r')
#os.chdir(r'C:\Users\danip\OneDrive\Desktop\IST-vibration\Data\coffe_dataset_v1')
#file_idle=open("data0.txt",'r')
#file_shake=open("data1.txt",'r')
idle_data_raw=file_idle.readlines()
shake_data_raw=file_shake.readlines()
idle_float,med_idle_x,med_idle_y,med_idle_z=process_raw(idle_data_raw)
shake_float,med_shake_x,med_shake_y,med_shake_z=process_raw(shake_data_raw)
idle_V=[]
shake_V=[]
idle_float_offset=[]
shake_float_offset=[]
filter_mode=input("Filter signals?(y/n):")
if filter_mode=='y':
    filter_frequency=float(input("Enter filter frequency:"))
    for line in idle_float:
        idle_V.append(math.sqrt(line[0]*line[0]+line[1]*line[1]+line[2]*line[2]))
    for line in shake_float:
        shake_V.append(math.sqrt(line[0]*line[0]+line[1]*line[1]+line[2]*line[2]))
    b,a=scipy.signal.butter(1,2*filter_frequency/26,btype='high')
    print("a:",a)
    print("b:",b)
    idle_V=scipy.signal.filtfilt(b,a,idle_V)
    shake_V=scipy.signal.filtfilt(b,a,shake_V)
if filter_mode=='n':
    offset_mode=input("Take off indle offset?(y/n):")
    if offset_mode=='y':
        for line in idle_float:
            new_line=[line[0]-med_idle_x,line[1]-med_idle_y,line[2]-med_idle_z]
            idle_float_offset.append(new_line)
        for line in shake_float:
            new_line=[line[0]-med_idle_x,line[1]-med_idle_y,line[2]-med_idle_z]
            shake_float_offset.append(new_line)
        with open("offset_idle.txt",'w') as idle_file:
            for line in idle_float:
                idle_file.write(" ".join(str(numb) for numb in line if abs(numb)>1))
                idle_file.write('\n')
        with open("offset_shake.txt",'w') as idle_file:
            for line in shake_float:
                idle_file.write(" ".join(str(numb) for numb in line if abs(numb)>1))
                idle_file.write('\n')
        for line in idle_float_offset:
            idle_V.append(math.sqrt(line[0]*line[0]+line[1]*line[1]+line[2]*line[2]))
        for line in shake_float_offset:
            shake_V.append(math.sqrt(line[0]*line[0]+line[1]*line[1]+line[2]*line[2]))
    else:
        for line in idle_float:
            idle_V.append(math.sqrt(line[0]*line[0]+line[1]*line[1]+line[2]*line[2]))
        for line in shake_float:
            shake_V.append(math.sqrt(line[0]*line[0]+line[1]*line[1]+line[2]*line[2]))
plt.plot(shake_V,label="shake")
plt.plot(idle_V,label="idle")
plt.title("Data")
plt.legend()
plt.show()

#features section

window_size=int(input("Enter window size:"))
user_mode=input("Enter desired feature(P,V,E):")
if user_mode=='E':
    shake_features=energy(shake_V,window_size)
    idle_features=energy(idle_V,window_size)
elif user_mode=='V':
    shake_features=var(shake_V,window_size)
    idle_features=var(idle_V,window_size)
else:
    shake_features=p2p(shake_V,window_size)
    idle_features=p2p(idle_V,window_size)
plt.figure()
plt.plot(shake_features,label="shake")
plt.plot(idle_features,label="idle")
plt.legend()
plt.title("Features in plot")
plt.figure()
plt.hist(shake_features,label='shake')
plt.hist(idle_features,label='idle')
plt.title("Features in histogram")
plt.legend()
plt.show()

This part is to analize result files in which there are 2 additional values in lines: the predicted result and ground truth. Most of the logic is identical to the previous code. The numenclature of the files might change as this project goes on but so far it is as such:

From what machine the MLC is built on to what machine it is experemented on. Ex:motor_to_coffe -> trained on motor data and experemented on the coffe machine.

E/V/P-> what feature it uses. So far it will only be used one feature but as this project goes on more might be used.
F-> indicated if there is a filter being used. If there is, so far, its a HBP filter with low cutoff frequencies (1 or 0.1 Hz) to remove DC component.

As a very short sidenote, the file motor_to_coffe_EF.txt has some misslabeling, namely in the transitions between idle and shake, so the results from that file are very preliminary.

In [ ]:
os.chdir(init_dir)
os.chdir(r'..\Data\results')
#file_result=open("motor_to_motor_EF_up.txt",'r')
#file_result=open("motor_to_motor_EF.txt",'r')
file_result=open("data0.txt",'r')
#file_result=open("data0.txt",'r')
result_data_raw=file_result.readlines()
result_float,med_res_x,med_res_y,med_res_z=process_raw(result_data_raw)
time=[x/330 for x in range(len(result_float))]
TP=0
TN=0
FP=0
FN=0
print("          SHAKE     IDLE")
for line in result_float:
    if line[3]==1 and line[4]==1:
        TP+=1
    elif line[3]==1 and line[4]==0:
        FP+=1
    elif line[3]==0 and line[4]==0:
        TN+=1
    elif line[3]==0 and line[4]==1:
        FN+=1
print("SHAKE     ",TP,"     ",FP)
print("IDLE      ",FN,"     ",TN)
print("Accuracy:",(TP+TN)/(TP+TN+FP+FN))

print("Precision:",TP/(TP+FP))
print("Recall:",TP/(TP+FN))
result_V=[]
for line in result_float:
    x=line[0]/2
    y=line[1]/2
    z=line[2]/2
    result_V.append(math.sqrt(x*x+y*y+z*z))
filter_mode=input("Filter signals?(y/n):")
if filter_mode=='y':
    filter_frequency=float(input("Enter filter frequency:"))
    b,a=scipy.signal.butter(1,2*filter_frequency/26,btype='high')
    print("a:",a)
    print("b:",b)
    result_V=scipy.signal.filtfilt(b,a,result_V)
GT=[]
Pred=[]
delay=[]
prev_GT=result_float[0][4]
delay_flag=0
delay_init=0
for index,line in enumerate(result_float):
    GT.append(line[4])
    Pred.append(line[3])
    if line[4]!=prev_GT:
        delay_flag=1
        delay_init=index
    prev_GT=line[4]
    if delay_flag==1:
        if line[3]==prev_GT:
            delay.append((index-delay_init)/330)
            delay_flag=0
print("delays:",delay)

window_size=10
filter_feautures=input("Filter features?(y/n):")
result_features=energy(result_V,window_size)
if filter_feautures=='y':
    result_features=[i for i in result_features if i<2000]

df=pd.DataFrame(dict(GroundTruth=GT,Prediction=Pred))
fig=ex.line(y=result_V)
fig.show("notebook")
fig1=ex.line(y=result_features)
fig1.show("notebook")
fig2=ex.line(df)
fig2.show("notebook")

With the MLC analized, the part of this project is to evaluate, compare and study some more intracate classifiers. For this, the first step is to analize the frequency response of the signal since it has aloth of very usefull information and according to the studies done previously, it shows that this domain is crutial to making a good classifier.

This next section is dedicated to analizing the FFT of the desired signal. This will have both the FFT built in feature of numpy and also the FFT done by the approxFFT for comparation of this algorith.

Additionaly, the peaks are calculated as well and compared with the librosa algorihm.

Its important to note that the max frequency is the nyquist frequency which is half of the sampling rate. 

In [ ]:

# sampling rate
sr = 6666

file_name="state24"
sample_view=6
axis=1
peaks_to_show=3

os.chdir(init_dir)
os.chdir(r'..\Data\corrected_new_motor\Features')
feature_file=open(file_name+"_Feature.txt",'r')
result_data_raw=feature_file.readlines()
#these are all of the features done by the aproxFFT algorithm
FFT_Dataset,peaks,peak_amp,mean_std=process_features(result_data_raw,20)

# sampling interval
ts = 1.0/sr
t = np.arange(0,1,ts)
N = len(FFT_Dataset[0][sample_view])
n = np.arange(N)
T = N/sr
freq = n/(T) 

X=FFT_Dataset[0][sample_view][:len(FFT_Dataset[0][sample_view])//2]
Y=FFT_Dataset[1][sample_view][:len(FFT_Dataset[1][sample_view])//2]
Z=FFT_Dataset[2][sample_view][:len(FFT_Dataset[2][sample_view])//2]


#now the numpy fft comparation

os.chdir(init_dir)
os.chdir(r'..\Data\corrected_new_motor')
feature_file=open(file_name+".txt",'r')
result_data_raw=feature_file.readlines()
float_list=process_raw(result_data_raw)
raw_Dataset=np.zeros((3),dtype=object)
for i in range(3):
    raw_Dataset[i]=[]
i=0
x_raw=[]
y_raw=[]
z_raw=[]
for line in float_list[0]:
    if(i<N):
        x_raw.append(line[0]+15)
        y_raw.append(line[1]+1000)
        z_raw.append(line[2]+105)
        i+=1
    else:
        raw_Dataset[0].append(x_raw)
        raw_Dataset[1].append(y_raw)
        raw_Dataset[2].append(z_raw)
        i=0
        x_raw=[]
        y_raw=[]
        z_raw=[]
    
window=np.hamming(N)
X_np=fft(raw_Dataset[0][sample_view]*window)
Y_np=fft(raw_Dataset[1][sample_view]*window)
Z_np=fft(raw_Dataset[2][sample_view]*window)
X_np[0]=0
Y_np[0]=0
Z_np[0]=0
freq_np=fftfreq(len(raw_Dataset[0][sample_view]),1/sr)
freq_np=freq_np[:len(freq_np)//2]
fig = make_subplots(rows=3, cols=2,shared_xaxes=True,vertical_spacing=0.08,subplot_titles=("Numpy FFT","ApproxFFT"))
fig.add_trace(go.Scatter(x=freq_np, y=abs(X_np),name="x_axis"),row=1, col=1)
fig.add_trace(go.Scatter(x=freq_np, y=abs(Y_np),name="y_axis"),row=2, col=1)
fig.add_trace(go.Scatter(x=freq_np, y=abs(Z_np),name="z_axis"),row=3, col=1)
fig.add_trace(go.Scatter(x=freq, y=X,name="x_axis"),row=1, col=2)
fig.add_trace(go.Scatter(x=freq, y=Y,name="y_axis"),row=2, col=2)
fig.add_trace(go.Scatter(x=freq, y=Z,name="z_axis"),row=3, col=2)

fig.show()
if axis==2:
    axis_to_analize=Z
elif axis==1:
    axis_to_analize=Y
else:
    axis_to_analize=X
peaks_comp=librosa.util.peak_pick(np.array(axis_to_analize),pre_max=len(axis_to_analize)//5, post_max=len(axis_to_analize)//5, pre_avg=3, post_avg=5, delta=0.5, wait=0)
fig = make_subplots(rows=1, cols=2,shared_xaxes=True,vertical_spacing=0.08,subplot_titles=("Peaks from Zscore","Peaks from librosa"))
fig.add_trace(go.Scatter(x=freq, y=axis_to_analize,name="FFT"),row=1,col=1)
fig.add_trace(go.Scatter(x=freq, y=axis_to_analize,name="FFT"),row=1,col=2)
i=0
for vals in peaks[axis][sample_view]:
    if(vals>0):
        fig.add_vline(x=vals, line_width=3, line_dash="dash", line_color="green",row=1,col=1)
        i+=1
for vals in peaks_comp:
    vals=vals/T
    fig.add_vline(x=vals, line_width=3, line_dash="dash", line_color="green",row=1,col=2)
fig.show("notebook")




The following sections are dedicated to utilizing ML algorithms to attempt to classify the dataset. The following algorithms are utilized:
* Kmeans
* Decision tree
* Random Forest
* Auto encoder
* SVM
* PCA

The Datasets utlized have 3 features available from their feature files:
* Complete FFT
* Peak values
* Peak amplitudes

These features are organized via the "build_dataset" function that returns these 3 features in seperate matrix. They are organized in the following way:

class -> axis -> feature

Each section has at the begining most of the inputs required for choosing features/defining parameters.

This section is dedicated to Random Forest. This is an algorithm that requires little preprocessing and is very efficient on the edge device itself, both in memory and execution time.

This algorithm is particularly good at detecting the most important features which can be a great insight for both its utilization and of other algorithms.

Sklearn page:https://scikit-learn.org/stable/modules/ensemble.html#forest

In [ ]:
os.chdir(init_dir)
#os.chdir(r'..\Data\CNC\Features')
os.chdir(r'..\Data\CNC\Features')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.inspection import permutation_importance

#This section has all of the configuration of the kmeans. peaks_or_fft define which feature will be used:0->FFT,1->peaks
#n_of_peaks_to_use defines the number of peaks to use if the samples are peaks.
#dataset_ration defines the size(%) of the test set.

peaks_or_fft=1
dataset_ratio=0.33
n_of_peaks_to_use=3
use_of_amps=1
sort=0
add_energy=1
add_mean=0
add_std=1
n_of_samples_per_class=2000

all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
mean_std=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n],mean_std[class_n]=process_features(result_data_raw,n_of_samples_per_class)

X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std)

clf=RandomForestClassifier(n_estimators=5)
clf.fit(X_train,y_train)
result = permutation_importance(
    clf, X_test, y_test, n_repeats=1000, random_state=42, n_jobs=2
)
print(result.importances_mean)
print(X_train[0])
"""
feature_names=["x_1","x_1_amp","x1_amp_scaled","x_2","x_2_amp","x2_amp_scaled","x_3","x_3_amp","x3_amp_scaled","x_energy","x_std","y_1","y_1_amp","y1_amp_scaled","y_2","y_2_amp","y2_amp_scaled","y_3","y_3_amp","y3_amp_scaled","y_energy","y_std","z_1","z_1_amp","z1_amp_scaled","z_2","z_2_amp","z2_amp_scaled","z_3","z_3_amp","z3_amp_scaled","z_energy","z_std"]
forest_importances = pd.Series(result.importances_mean, index=feature_names)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()
N=3
res = sorted(range(len(forest_importances)), key = lambda sub: forest_importances[sub])[-N:]
"""
test_labels=clf.predict(X_test)
cm=confusion_matrix(y_test,test_labels)
print(clf.score(X_test,y_test))
DT=clf.estimators_[0]
print(clf.estimators_[0].get_depth())
print(clf.estimators_[0].get_n_leaves())
print(cm)
max_peaks=[0,0,0]
for label in peaks:
    for axis_idx,axis in enumerate(label):
        for sample in axis:
            for i in range(n_of_peaks_to_use):
                if sample[i]>max_peaks[axis_idx]:
                    max_peaks[axis_idx]=sample[i]
print(max_peaks)

x_axis=[]
y_axis=[]
z_axis=[]
labels=[]
unique,count=np.unique(y_train,return_counts=True)
n_of_training_samples=count[0]
print(len(X_train[0]))
for j in range(len(all_files)):
    for i in range(100):
        x_axis.append(X_train[i+j*n_of_training_samples][res[0]])
        y_axis.append(X_train[i+j*n_of_training_samples][res[1]])
        z_axis.append(X_train[i+j*n_of_training_samples][res[2]])
        labels.append(y_train[i+j*n_of_training_samples])
data={'x_axis':x_axis,'y_axis':y_axis,'z_axis':z_axis,'labels':labels}
df=pd.DataFrame(data)
fig = px.scatter_3d(df, x='x_axis', y='y_axis', z='z_axis',
            color='labels')
fig.update_layout(height=600, width=600,title_text="points of data")
fig.show()

This section is dedicated to building the SVM model. It utilized the SVC classifier from sklearn.

SVM has proven to give good results, however, from the tests done in this section it has shown to be less efficient both in memory (around 100x bigger) and in execution time. 

Sklearn page:https://scikit-learn.org/stable/modules/svm.html

In [ ]:
from sklearn import svm
from micromlgen import port
import sys
os.chdir(init_dir)
os.chdir(r'..\Data\actual_new_motor\Features')
from sklearn.metrics import confusion_matrix


#This section has all of the configuration of the kmeans. peaks_or_fft define which feature will be used:0->FFT,1->peaks
#n_of_peaks_to_use defines the number of peaks to use if the samples are peaks.
#dataset_ration defines the size(%) of the test set.

peaks_or_fft=1
dataset_ratio=0.70
n_of_peaks_to_use=3
use_of_amps=0
sort=0

all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n]=process_features(result_data_raw)
X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,0)
clf=svm.SVC(gamma=1 / (9 * np.var(X_train[0])))
clf.fit(X_train,y_train)
test_labels=clf.predict(X_test)
cm=confusion_matrix(y_test,test_labels)
print(clf.score(X_test,y_test))
print(type(clf.support_vectors_[0][0]))
print(sys.getsizeof(clf.support_vectors_))
print(cm)
os.chdir(init_dir)
os.chdir(r'..\Data\actual_new_motor\Models')
with open("SVC_model.h",'w') as file:
    file.write(port(clf))
    file.close()
new_svm=[]
for vector_idx,vector in enumerate(clf.support_vectors_):
    quantized_vector=[]
    for val in vector:
        quantized_vector.append(int(val))
    new_svm.append(quantized_vector)
test_labels=clf.predict(X_test)
cm=confusion_matrix(y_test,test_labels)
print(clf.score(X_test,y_test))
clf.support_vectors_=new_svm
print(type(clf.support_vectors_[0][0]))
print(sys.getsizeof(clf.support_vectors_))
print(len(clf.support_vectors_[0]))
print(cm)
with open("SVC_model_Q.h",'w') as file:
    file.write(port(clf))
    file.close()


This section is dedicated to making the Decision tree model. 

In general, if this section has a performance as good as RF, it is recomended to use this model instead since it is more efficient since RF is a collection of DT therefore it is less efficient. This is by no means a hard rule and both algorithms should be analized before deployment.

Sklearn page:https://scikit-learn.org/stable/modules/tree.html#tree

In [ ]:
os.chdir(init_dir)
os.chdir(r'..\Data\corrected_new_motor\Features')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn.tree import DecisionTreeClassifier
from micromlgen import port

#This section has all of the configuration of the kmeans. peaks_or_fft define which feature will be used:0->FFT,1->peaks
#n_of_peaks_to_use defines the number of peaks to use if the samples are peaks.
#dataset_ration defines the size(%) of the test set.

peaks_or_fft=1
dataset_ratio=0.33
n_of_peaks_to_use=3
use_of_amps=1
sort=1
add_energy=1
add_mean=1
add_std=1
n_of_samples_per_class=-1

all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
mean_std=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n],mean_std[class_n]=process_features(result_data_raw,n_of_samples_per_class)

X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std)

clf=DecisionTreeClassifier(random_state=0)
path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
print(X_train[0])
print(impurities)
clf.fit(X_train,y_train)
test_labels=clf.predict(X_test)
cm=confusion_matrix(y_test,test_labels)
print(clf.score(X_test,y_test))
print(cm)
os.chdir(r'..\Models')
with open("DT_model.h",'w') as file:
    file.write(port(clf))
    file.close()
max_score=0
max_score_cpp=0
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    test_labels=clf.predict(X_test)
    score=clf.score(X_test,y_test)
    if score>max_score:
        max_score=score
        max_score_cpp=ccp_alpha
clf = DecisionTreeClassifier(random_state=0, ccp_alpha=max_score_cpp)
clf.fit(X_train, y_train)
test_labels=clf.predict(X_test)
cm=confusion_matrix(y_test,test_labels)
print(clf.score(X_test,y_test))
print(cm)
with open("DT_model_P.h",'w') as file:
    file.write(port(clf))
    file.close()
values=clf.tree_.threshold
new_tree_threshold=[]
for val in values:
    new_val=int(val)
    new_tree_threshold.append(new_val)


This section is dedicated to K-means. K-means is a noteworthy algorithm because it is unsupervised and is able to define areas of confidence in which, if a sample is outside of it, it can be considered an anomaly or a new state. This can possibly allow for an on training aproach on the edge device.

 K-means done directly has poor results generaly but by choosing important features it can have decent results. This sections acts as an almost preliminary section for K-means as this algorithm will be expanded by the addition of others.

 Sklearn page:https://scikit-learn.org/stable/modules/clustering.html#k-means

In [ ]:
os.chdir(init_dir)
os.chdir(r'..\Data\corrected_new_motor\Features')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn import preprocessing


#This section has all of the configuration of the kmeans. FFT/PEAKS define what samples will be used. Please assign 1 to the desired and 0 to the other.
#n_of_peaks_to_use defines the number of peaks to use if the samples are peaks.
#dataset_ration defines the size(%) of the test set.

peaks_or_fft=1
dataset_ratio=0.33
n_of_peaks_to_use=3
use_of_amps=1
sort=1
add_energy=0
add_mean=0
add_std=0
n_of_samples_per_class=-1

all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
mean_std=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n],mean_std[class_n]=process_features(result_data_raw,n_of_samples_per_class)

X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std)

"""
fig=make_subplots(rows=1,cols=3,shared_xaxes=True,vertical_spacing=0.08,subplot_titles=("X axis","Y axis","Z axis"))
x_peaks=[]
y_peaks=[]
z_peaks=[]
labels=[]
i=0
for class_n,all_peaks_in_class in enumerate(peaks):
    for peak_index in range(50):
        x_peaks.append(peaks[class_n][0][peak_index])
        y_peaks.append(peaks[class_n][1][peak_index])
        z_peaks.append(peaks[class_n][2][peak_index])
        labels.append(class_n)
    for vals in range(50):
        x_peaks[vals+i]=sorted(x_peaks[vals+i])
        y_peaks[vals+i]=sorted(y_peaks[vals+i])
        z_peaks[vals+i]=sorted(z_peaks[vals+i])
    i+=50

fig.add_trace(go.Scatter(x=[item[0] for item in x_peaks],y=[item[1] for item in x_peaks],mode="markers+text",marker=dict(color=labels)),row=1,col=1)
fig.add_trace(go.Scatter(x=[item[0] for item in y_peaks],y=[item[1] for item in y_peaks],mode="markers+text",marker=dict(color=labels)),row=1,col=2)
fig.add_trace(go.Scatter(x=[item[0] for item in z_peaks],y=[item[1] for item in z_peaks],mode="markers+text",marker=dict(color=labels)),row=1,col=3)
fig.show("notebook")
"""
"""
new_X_train=[]
for sample in X_train:
    new_sample=[sample[0],sample[7],sample[8]]
    new_X_train.append(new_sample)
#X_train=new_X_train
new_X_test=[]
for sample in X_test:
    new_sample=[sample[0],sample[7],sample[8]]
    new_X_test.append(new_sample)
#X_test=new_X_test
"""
WCSS=[]
for i in range(1,15):
    clf=KMeans(n_clusters=i,init='k-means++',random_state=42)
    clf.fit(X_train,y_train)
    WCSS.append(clf.inertia_)
fig=px.scatter(x=range(1,15),y=WCSS)
fig.show()


kmeans=KMeans(n_clusters=8,random_state=0,n_init="auto",init='k-means++')
kmeans.fit(X_train)
test_labels=kmeans.predict(X_test)

if(n_of_peaks_to_use==1 and peaks_or_fft==1):
    x_axis=[]
    y_axis=[]
    z_axis=[]
    labels=[]
    unique,count=np.unique(y_train,return_counts=True)
    n_of_training_samples=count[0]
    for j in range(len(all_files)):
        for i in range(100):
            x_axis.append(X_train[i+j*n_of_training_samples][0])
            y_axis.append(X_train[i+j*n_of_training_samples][1])
            z_axis.append(X_train[i+j*n_of_training_samples][2])
            labels.append(y_train[i+j*n_of_training_samples])
    data={'x_axis':x_axis,'y_axis':y_axis,'z_axis':z_axis,'labels':labels}
    df=pd.DataFrame(data)
    fig = px.scatter_3d(df, x='x_axis', y='y_axis', z='z_axis',
                color='labels')
    fig.update_layout(height=600, width=600,title_text="points of data")
    fig.show()
    x_axis=[]
    y_axis=[]
    z_axis=[]
    labels=[]
    for i in range(len(X_test)):
        x_axis.append(X_test[i][0])
        y_axis.append(X_test[i][1])
        z_axis.append(X_test[i][2])
        labels.append(test_labels[i])
    data={'x_axis':x_axis,'y_axis':y_axis,'z_axis':z_axis,'labels':labels}
    df=pd.DataFrame(data)
    fig = px.scatter_3d(df, x='x_axis', y='y_axis', z='z_axis',
                color='labels')
    fig.update_layout(height=600, width=600,title_text="test things")
    fig.show()
    x_axis=[]
    y_axis=[]
    z_axis=[]
    labels=[]
    for i in range(7):
        x_axis.append(kmeans.cluster_centers_[i][0])
        y_axis.append(kmeans.cluster_centers_[i][1])
        z_axis.append(kmeans.cluster_centers_[i][2])
        labels.append(i)
    data={'x_axis':x_axis,'y_axis':y_axis,'z_axis':z_axis,'labels':labels}
    df=pd.DataFrame(data)
    fig = px.scatter_3d(df, x='x_axis', y='y_axis', z='z_axis',color='labels')
    fig.update_layout(height=600, width=600,title_text="cluster")
    fig.show()

cm=confusion_matrix(y_test,test_labels)
print(X_train[0])
print(cm)


This section is dedicated to utilizing PCA+Kmeans. Performing PCA before can help make clusters clearer,making K-means perform better, however, it requires computation time and memory and these should be taken into account.

Additionaly, it can be used to better understand relevant features. For example, for the "actual_new_motor" dataset, energy is a great feature and greatly impacts the PCA+K-means.

Sklearn page:https://scikit-learn.org/stable/modules/decomposition.html#pca

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn import preprocessing
from micromlgen import port
os.chdir(init_dir)
os.chdir(r'..\Data\CNC\Features')



#This section has all of the configuration of the kmeans. FFT/PEAKS define what samples will be used. Please assign 1 to the desired and 0 to the other.
#n_of_peaks_to_use defines the number of peaks to use if the samples are peaks.
#dataset_ration defines the size(%) of the test set.

peaks_or_fft=0
dataset_ratio=0.33
n_of_peaks_to_use=3
use_of_amps=1
sort=1
add_energy=1
add_mean=0
add_std=1
n_of_samples_per_class=2000

all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
mean_std=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n],mean_std[class_n]=process_features(result_data_raw,n_of_samples_per_class)

X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std)

fig=make_subplots(rows=1,cols=3,shared_xaxes=True,vertical_spacing=0.08,subplot_titles=("X axis","Y axis","Z axis"))
x_peaks=[]
y_peaks=[]
z_peaks=[]
labels=[]
i=0
for class_n,all_peaks_in_class in enumerate(peaks):
    for peak_index in range(50):
        x_peaks.append(peaks[class_n][0][peak_index])
        y_peaks.append(peaks[class_n][1][peak_index])
        z_peaks.append(peaks[class_n][2][peak_index])
        labels.append(class_n)
    for vals in range(50):
        x_peaks[vals+i]=sorted(x_peaks[vals+i])
        y_peaks[vals+i]=sorted(y_peaks[vals+i])
        z_peaks[vals+i]=sorted(z_peaks[vals+i])
    i+=50
os.chdir(r'..\Models')
if(peaks_or_fft==0):
    final_dimentionality=25
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_pre_pca = scaler.transform(X_train)
    X_test_pre_pca = scaler.transform(X_test)
    pca = PCA(n_components=final_dimentionality)
    pca.fit(X_train_pre_pca)
    X_train_pca=pca.transform(X_train_pre_pca)
    X_test_pca=pca.transform(X_test_pre_pca)
    if(final_dimentionality==3):
        x_axis=[]
        y_axis=[]
        z_axis=[]
        labels=[]
        unique,count=np.unique(y_train,return_counts=True)
        n_of_training_samples=count[0]
        for j in range(len(all_files)):
            for i in range(100):
                x_axis.append(X_train_pca[i+j*n_of_training_samples][0])
                y_axis.append(X_train_pca[i+j*n_of_training_samples][1])
                z_axis.append(X_train_pca[i+j*n_of_training_samples][2])
                labels.append(y_train[i+j*n_of_training_samples])
        data={'x_axis':x_axis,'y_axis':y_axis,'z_axis':z_axis,'labels':labels}
        df=pd.DataFrame(data)
        fig = px.scatter_3d(df, x='x_axis', y='y_axis', z='z_axis',
                    color='labels')
        fig.update_layout(height=600, width=600,title_text="points of data")
        fig.show()
    X_train=X_train_pca
    X_test=X_test_pca
    port_scaller(scaler,"Scaler_Q",1)
    port_scaller(scaler,"Scaler",0)
    with open("PCA.h",'w') as file:
        file.write(port(pca))

WCSS=[]
for i in range(1,15):
    clf=KMeans(n_clusters=i,init='k-means++',random_state=42)
    clf.fit(X_train,y_train)
    WCSS.append(clf.inertia_)
fig=px.scatter(x=range(1,15),y=WCSS)
fig.show()
kmeans=KMeans(n_clusters=8,init='k-means++',random_state=32)
kmeans.fit(X_train)
test_labels=kmeans.predict(X_test)
cm=confusion_matrix(y_test,test_labels)
print(cm)
port_kmeans_to_C(kmeans,"Kmeans_Q",X_train,1)
port_kmeans_to_C(kmeans,"Kmeans",X_train,0)

This section is dedicated to autoencoder+Kmeans. Autoencoders, just like PCA, can reduce the dimention of the data and in turn represent it better for clustering, however, it requires fine tunning unlike PCA.

From the analises done at the time of this writing, autoencoders seem to have a poor performance and cannot represent the data correctly, however, it can be a great tool if made to work since it, by itself, can detect anomalies/new classes without the need for any computation done on K-means.

Keras page:https://blog.keras.io/building-autoencoders-in-keras.html

In [ ]:
os.chdir(init_dir)
os.chdir(r'..\Data\actual_new_motor\Features')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from tensorflow import keras
from sklearn import preprocessing



#This section has all of the configuration of the kmeans. FFT/PEAKS define what samples will be used. Please assign 1 to the desired and 0 to the other.
#n_of_peaks_to_use defines the number of peaks to use if the samples are peaks.
#dataset_ration defines the size(%) of the test set.

peaks_or_fft=1
dataset_ratio=0.33
n_of_peaks_to_use=3
use_of_amps=1
sort=1
add_energy=1


all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n]=process_features(result_data_raw,-1)
fig=make_subplots(rows=1,cols=3,shared_xaxes=True,vertical_spacing=0.08,subplot_titles=("X axis","Y axis","Z axis"))
x_peaks=[]
y_peaks=[]
z_peaks=[]
labels=[]
i=0
for class_n,all_peaks_in_class in enumerate(peaks):
    for peak_index in range(50):
        x_peaks.append(peaks[class_n][0][peak_index])
        y_peaks.append(peaks[class_n][1][peak_index])
        z_peaks.append(peaks[class_n][2][peak_index])
        labels.append(class_n)
    for vals in range(50):
        x_peaks[vals+i]=sorted(x_peaks[vals+i])
        y_peaks[vals+i]=sorted(y_peaks[vals+i])
        z_peaks[vals+i]=sorted(z_peaks[vals+i])
    i+=50
X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_pre_ae = scaler.transform(X_train)
X_test_pre_ae = scaler.transform(X_test)
input_size = (n_of_peaks_to_use+use_of_amps*n_of_peaks_to_use+add_energy)*3
encoding_dim= 3
input_layer = keras.layers.Input(shape=(input_size,))
encoding_layer=keras.layers.Dense(encoding_dim,activation='relu')(input_layer)
decoding_layer=keras.layers.Dense(input_size,activation='relu')(encoding_layer)
autoencoder=keras.Model(input_layer,decoding_layer)
encoder=keras.Model(input_layer,encoding_layer)
autoencoder.compile(optimizer='adam',loss='mean_squared_error')
X_train_input=np.array([np.array(xi) for xi in X_train_pre_ae])
X_test_input=np.array([np.array(xi) for xi in X_test_pre_ae])
autoencoder.fit(X_train_input,X_train_input,epochs=100,batch_size=256,validation_data=(X_test_input,X_test_input))
X_test_pred=autoencoder.predict(X_test_input, batch_size=32, verbose="auto", steps=None, callbacks=None)
X_test_autoencoder=encoder.predict(X_test_input, batch_size=32, verbose="auto", steps=None, callbacks=None)
X_train_autoencoder=encoder.predict(X_train_input, batch_size=32, verbose="auto", steps=None, callbacks=None)
x_axis=[]
y_axis=[]
z_axis=[]
labels=[]
unique,count=np.unique(y_train,return_counts=True)
n_of_training_samples=count[0]

for j in range(len(all_files)):
    for i in range(100):
        x_axis.append(X_train_autoencoder[i+j*n_of_training_samples][0])
        y_axis.append(X_train_autoencoder[i+j*n_of_training_samples][1])
        z_axis.append(X_train_autoencoder[i+j*n_of_training_samples][2])
        labels.append(y_train[i+j*n_of_training_samples])
data={'x_axis':x_axis,'y_axis':y_axis,'z_axis':z_axis,'labels':labels}
df=pd.DataFrame(data)
fig = px.scatter_3d(df, x='x_axis', y='y_axis', z='z_axis',
            color='labels')
fig.update_layout(height=600, width=600,title_text="points of data")
fig.show()

kmeans=KMeans(n_clusters=len(peaks),random_state=0,n_init="auto")
kmeans.fit(X_train_autoencoder)
test_labels=kmeans.predict(X_test_autoencoder)
cm=confusion_matrix(y_test,test_labels)
print(cm)


This section is dedicated to isolation trees(IT). Isolation trees are similar to DT in structure, however, instead of attempting to classify the dataset, it will perform outlier detection by making sure non outliers run throught most the the tree while outliers stop very close to the initial node. The logic behind using IT for classification is to provide an unsupervised tree based aproach. All classes will have their own IT and one of them will have a non outlier while all of the other will have that sample classified as an outlier.

In [ ]:
os.chdir(init_dir)
os.chdir(r'..\Data\CNC\Features')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn import preprocessing
from isotree import IsolationForest
from sklearn.decomposition import IncrementalPCA

import sys

peaks_or_fft=1
dataset_ratio=0.7
n_of_peaks_to_use=3
use_of_amps=1
sort=1
add_energy=1
add_mean=0
add_std=1
n_of_samples_per_class=2000

all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
mean_std=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n],mean_std[class_n]=process_features(result_data_raw,n_of_samples_per_class)

X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std)

X_train_IT=[]
X_test_IT=[]
y_test_IT=[]
current_state=-1
X_train_aux=[]
test=0
for sample_n,sample in enumerate(X_train):
    if(current_state==-1):
        current_state=y_train[sample_n]
    if(y_train[sample_n]==current_state):
        X_train_aux.append(sample)
    else:
        X_train_IT.append(X_train_aux)
        X_train_aux=[]
    current_state=y_train[sample_n]
X_train_IT.append(X_train_aux)

current_state=-1
X_test_aux=[]
y_test_aux=[]
for sample_n,sample in enumerate(X_test):
    if(current_state==-1):
        current_state=y_test[sample_n]
    if(y_test[sample_n]==current_state):
        X_test_aux.append(sample)
        y_test_aux.append(y_test[sample_n])
    else:
        X_test_IT.append(X_test_aux)
        y_test_IT.append(y_test_aux)
        X_test_aux=[]
        y_test_aux=[]
    current_state=y_test[sample_n]
X_test_IT.append(X_test_aux)
y_test_IT.append(y_test_aux)

clf=[]
for all_state_samples in X_train_IT:
    temp_clf=IsolationForest(
    ndim=2, ntrees=100,
    missing_action="fail"
)
    temp_clf.fit(np.array(all_state_samples))
    clf.append(temp_clf) 

       
res=[]
corr=0
err=0
test=[]
for state,all_samples_in_state in enumerate(X_test_IT):
    for sample_n,sample in enumerate(all_samples_in_state):
        final_res=0
        scores=[]
        min_score=999
        for IT_class,ITs in enumerate(clf):
            sample_to_pred=np.array(sample)
            pred=ITs.predict(sample_to_pred.reshape(1,-1))
            scores.append(pred)
            if pred<min_score:
                final_res=IT_class
                min_score=pred
            if IT_class==4 and state==3:
                test.append([3,4,pred])
            if IT_class==3 and state==3:
                test.append([3,3,pred])
        if min_score>0.60:
            final_res=-1
        res.append(final_res)
        if final_res==state:
            corr+=1
        else:
            err+=1
print(test)
print(corr)
print(err)
cm=confusion_matrix(y_test[:len(res)],res)
print("Accuracy:",corr/(corr+err)*100)
print(cm)

vals=[]
for it in clf:
    vals.append(it.predict(np.array(X_train_IT[4][0]).reshape(1,-1)))
print(vals)

print(X_train_IT[3][0])
print(X_train_IT[4][0])


This section will make the actual on training IT.

In [ ]:
os.chdir(init_dir)
os.chdir(r'..\Data\CNC\Features')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn import preprocessing
from isotree import IsolationForest
from sklearn.decomposition import IncrementalPCA
import sys

peaks_or_fft=1
dataset_ratio=0.33
n_of_peaks_to_use=6
use_of_amps=1
sort=1
add_energy=1
add_mean=0
add_std=1
n_of_samples_per_class=2000

all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
mean_std=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n],mean_std[class_n]=process_features(result_data_raw,n_of_samples_per_class)

X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std)

print(X_train[0])
n_of_sample_to_train=50
X_train_IT=[]
X_test_IT=[]
y_test_IT=[]
current_state=-1
X_train_aux=[]
test=0
for sample_n,sample in enumerate(X_train):
    if(current_state==-1):
        current_state=y_train[sample_n]
    if(y_train[sample_n]==current_state):
        X_train_aux.append(sample)
    else:
        X_train_IT.append(X_train_aux)
        X_train_aux=[]
    current_state=y_train[sample_n]
X_train_IT.append(X_train_aux)

current_state=-1
X_test_aux=[]
y_test_aux=[]
for sample_n,sample in enumerate(X_test):
    if(current_state==-1):
        current_state=y_test[sample_n]
    if(y_test[sample_n]==current_state):
        X_test_aux.append(sample)
        y_test_aux.append(y_test[sample_n])
    else:
        X_test_IT.append(X_test_aux)
        y_test_IT.append(y_test_aux)
        X_test_aux=[]
        y_test_aux=[]
    current_state=y_test[sample_n]
X_test_IT.append(X_test_aux)
y_test_IT.append(y_test_aux)

res=[]
clf=[]
max_score=[]
threshold=[]
switch_state=0
sample_to_train=[]
sample_to_train_label=[]

for state,all_samples_state in enumerate(X_train_IT):
    for sample in all_samples_state:
        if len(clf)==0:
            switch_state+=1
            sample_to_train.append(sample)
            sample_to_train_label.append(state)
        else:
            final_res=0
            scores=[]
            min_score=999
            for IT_class,ITs in enumerate(clf):
                pred=ITs.predict(np.array(sample).reshape(1,-1))
                scores.append(pred)
                if pred<min_score:
                    final_res=IT_class
                    min_score=pred
            if min_score>threshold[final_res]:
                final_res=-1
                switch_state+=1
                sample_to_train.append(sample)
                sample_to_train_label.append(state)
            elif switch_state>0:
                switch_state-=1
                sample_to_train.pop(0)
                sample_to_train_label.pop(0)
            res.append(final_res)
        if(switch_state==n_of_sample_to_train):
            temp_clf=IsolationForest(ndim=2, ntrees=100,
    missing_action="fail",sample_size=switch_state)
            temp_clf.fit(np.array(sample_to_train))
            final_train=[]
            test=0
            for sample in sample_to_train:
                val=temp_clf.predict(np.array(sample).reshape(1,-1))
                if val<0.6:
                    final_train.append(sample)
                else:
                    test+=1
            final_clf=IsolationForest(ndim=3, ntrees=200,
    missing_action="fail")
            final_clf.fit(np.array(final_train))
            max_score_pred=final_clf.predict(np.array(final_train))
            max_score_pred.sort()
            threshold.append(np.mean(max_score_pred)+1.5*np.std(max_score_pred))
            #threshold.append(0.6)
            clf.append(final_clf)
            print("Made new IT for state:",state)
            switch_state=0
            sample_to_train=[] 
            sample_to_train_label=[]
print("Done training")
res=[]
corr=0
err=0
merge_test=[0]*(len(clf)+1)
for state,all_samples_in_state in enumerate(X_test_IT):
    for sample_n,sample in enumerate(all_samples_in_state):
        final_res=0
        scores=[]
        min_score=999
        prev_res=-1
        for IT_class,ITs in enumerate(clf):
            pred=ITs.predict(np.array(sample).reshape(1,-1))
            scores.append(pred)
            if pred<min_score:
                prev_res=final_res
                final_res=IT_class
                min_score=pred
        if min_score>threshold[final_res]:
            final_res=-1
        if state==3:
            if final_res!=-1:
                merge_test[final_res]+=1
                merge_test[scores.index(sorted(scores)[1])]+=1
            else:
                merge_test[len(clf)]+=1
                merge_test[scores.index(sorted(scores)[0])]+=1

        res.append(final_res)
        if final_res==state:
            corr+=1
        else:
            err+=1
print(corr)
print(err)
cm=confusion_matrix(y_test[:len(res)],res)
print("Accuracy:",corr/(corr+err)*100)
print(cm)

print(threshold)
print(merge_test)




Kmeans+Incremental PCA

In [ ]:
os.chdir(init_dir)
os.chdir(r'..\Data\corrected_new_motor\Features')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn import preprocessing
from sklearn.decomposition import IncrementalPCA
import sys

peaks_or_fft=0
dataset_ratio=0.33
n_of_peaks_to_use=3
use_of_amps=1
sort=0
add_energy=1
add_mean=0
add_std=1
n_of_samples_per_class=-1

all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
mean_std=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n],mean_std[class_n]=process_features(result_data_raw,n_of_samples_per_class)

X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std)

X_train_IT=[]
X_test_IT=[]
y_test_IT=[]
current_state=-1
X_train_aux=[]
test=0
for sample_n,sample in enumerate(X_train):
    if(current_state==-1):
        current_state=y_train[sample_n]
    if(y_train[sample_n]==current_state):
        X_train_aux.append(sample)
    else:
        X_train_IT.append(X_train_aux)
        X_train_aux=[]
    current_state=y_train[sample_n]
X_train_IT.append(X_train_aux)

current_state=-1
X_test_aux=[]
y_test_aux=[]
for sample_n,sample in enumerate(X_test):
    if(current_state==-1):
        current_state=y_test[sample_n]
    if(y_test[sample_n]==current_state):
        X_test_aux.append(sample)
        y_test_aux.append(y_test[sample_n])
    else:
        X_test_IT.append(X_test_aux)
        y_test_IT.append(y_test_aux)
        X_test_aux=[]
        y_test_aux=[]
    current_state=y_test[sample_n]
X_test_IT.append(X_test_aux)
y_test_IT.append(y_test_aux)

clusters=[]
max_distances=[]
n_of_sample_to_train=100


ipca=IncrementalPCA(n_components=3)
scaler = preprocessing.StandardScaler().fit(X_train_IT[2])
ipca.partial_fit(np.array(scaler.transform(X_train_IT[2])))
kmeans_data=ipca.transform(np.array(scaler.transform(X_train_IT[2])))
kmeans=KMeans(n_clusters=1,init='k-means++',random_state=0)
kmeans.fit(kmeans_data)
new_cluster=[0]*3
for sample in kmeans_data:
    for feature_n,feature in enumerate(sample):
        new_cluster[feature_n]+=feature
for feature in range(3):
    new_cluster[feature]=new_cluster[feature]/len(kmeans_data)
print(new_cluster)
max_distance=0
distance_log=[]
for point in kmeans_data:
    distance=0
    for feature_n,feature in enumerate(point):
        distance=(feature-kmeans.cluster_centers_[0][feature_n])*(feature-kmeans.cluster_centers_[0][feature_n])
    distance=math.sqrt(distance)
    distance_log.append(distance)
distance_log.sort()
mean=np.mean(distance_log)
std=np.std(distance_log)
new_distance_log=[]
test=0
for distance in distance_log:
    if distance<mean+3*std:
        new_distance_log.append(distance)
    else:
        test+=1

print(test)
print(mean+3*std)
max_distance=new_distance_log[-1]
test_points=[]
for point in kmeans_data:
    distance=0
    for feature_n,feature in enumerate(point):
        distance+=(feature-kmeans.cluster_centers_[0][feature_n])*(feature-kmeans.cluster_centers_[0][feature_n])
    distance=math.sqrt(distance)
    if distance>max_distance:
        test_points.append(point)
print(max_distance)
kmeans_test=ipca.transform(np.array(scaler.transform(X_train_IT[3])))
test_corr=0
test_err=0
print(distance_log)
distance_log=[]
for point in kmeans_test:
    distance=0
    for feature_n,feature in enumerate(point):
        distance+=(feature-kmeans.cluster_centers_[0][feature_n])*(feature-kmeans.cluster_centers_[0][feature_n])
    distance=math.sqrt(distance)
    distance_log.append(distance)
    if distance>max_distance:
        test_corr+=1
    else:
        test_err+=1
distance_log.sort()
print(distance_log)
print(test_err)
print(test_corr)
print(kmeans.cluster_centers_)
x_axis=[]
y_axis=[]
z_axis=[]
label=[]
for i in range(len(kmeans_data)):
    x_axis.append(kmeans_data[i][0])
    y_axis.append(kmeans_data[i][1])
    z_axis.append(kmeans_data[i][2])
    label.append(0)
for i in range(len(kmeans_test)):
    x_axis.append(kmeans_test[i][0])
    y_axis.append(kmeans_test[i][1])
    z_axis.append(kmeans_test[i][2])
    label.append(1)
for i in range(len(test_points)):
    x_axis.append(test_points[i][0])
    y_axis.append(test_points[i][1])
    z_axis.append(test_points[i][2])
    label.append(2)
data={'x_axis':x_axis,'y_axis':y_axis,'z_axis':z_axis,'labels':label}
df=pd.DataFrame(data)
fig = px.scatter_3d(df, x='x_axis', y='y_axis', z='z_axis',color='labels')
fig.update_layout(height=600, width=600,title_text="points of data")
fig.show()

old_cluster=ipca.inverse_transform(kmeans.cluster_centers_.reshape(1,-1))


ipca.partial_fit(np.array(scaler.transform(X_train_IT[3])))



kmeans_pre_data=X_train_IT[2]
kmeans_pre_data.extend(X_train_IT[3])

new_cluster=ipca.transform(old_cluster)
kmeans_data=ipca.transform(np.array(scaler.transform(kmeans_pre_data)))
kmeans_show_1=ipca.transform(np.array(scaler.transform(X_train_IT[2])))
kmeans_show_2=ipca.transform(np.array(scaler.transform(X_train_IT[3])))

kmeans=KMeans(n_clusters=2,init='k-means++',random_state=0)
kmeans.fit(kmeans_data)
kmeans_test=ipca.transform(np.array(scaler.transform(X_train_IT[4])))
x_axis=[]
y_axis=[]
z_axis=[]
label=[]
for i in range(len(kmeans_show_1)):
    x_axis.append(kmeans_show_1[i][0])
    y_axis.append(kmeans_show_1[i][1])
    z_axis.append(kmeans_show_1[i][2])
    label.append(0)
for i in range(len(kmeans_show_2)):
    x_axis.append(kmeans_show_2[i][0])
    y_axis.append(kmeans_show_2[i][1])
    z_axis.append(kmeans_show_2[i][2])
    label.append(1)
for i in range(len(kmeans_test)):
    x_axis.append(kmeans_test[i][0])
    y_axis.append(kmeans_test[i][1])
    z_axis.append(kmeans_test[i][2])
    label.append(2)
data={'x_axis':x_axis,'y_axis':y_axis,'z_axis':z_axis,'labels':label}
df=pd.DataFrame(data)
fig = px.scatter_3d(df, x='x_axis', y='y_axis', z='z_axis',color='labels')
fig.update_layout(height=600, width=600,title_text="points of data")
fig.show()
print(kmeans.cluster_centers_)
print(new_cluster)


Kmeans+PCA on training.

In [ ]:
os.chdir(init_dir)
os.chdir(r'..\Data\actual_new_motor\Features')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn import preprocessing
from sklearn.decomposition import IncrementalPCA
import sys

peaks_or_fft=1
dataset_ratio=0.33
n_of_peaks_to_use=2
use_of_amps=1
sort=1
add_energy=1
add_mean=0
add_std=1
n_of_samples_per_class=-1

all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
mean_std=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n],mean_std[class_n]=process_features(result_data_raw,n_of_samples_per_class)

X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std)

X_train_IT=[]
X_test_IT=[]
y_test_IT=[]
current_state=-1
X_train_aux=[]
test=0
for sample_n,sample in enumerate(X_train):
    if(current_state==-1):
        current_state=y_train[sample_n]
    if(y_train[sample_n]==current_state):
        X_train_aux.append(sample)
    else:
        X_train_IT.append(X_train_aux)
        X_train_aux=[]
    current_state=y_train[sample_n]
X_train_IT.append(X_train_aux)


current_state=-1
X_test_aux=[]
y_test_aux=[]
for sample_n,sample in enumerate(X_test):
    if(current_state==-1):
        current_state=y_test[sample_n]
    if(y_test[sample_n]==current_state):
        X_test_aux.append(sample)
        y_test_aux.append(y_test[sample_n])
    else:
        X_test_IT.append(X_test_aux)
        y_test_IT.append(y_test_aux)
        X_test_aux=[]
        y_test_aux=[]
    current_state=y_test[sample_n]
X_test_IT.append(X_test_aux)
y_test_IT.append(y_test_aux)


clusters=[]
temp_new_clusters=[]
max_distances=[]
samples_to_train=[]
permanent_samples_for_train=[]
max_distance_sample=[]
batch_size=20
batch=[]
switch_state=0
n_of_samples_to_train=50
low_dimention=3
x_axis=[]
y_axis=[]
z_axis=[]
label=[]


ipca=IncrementalPCA(n_components=low_dimention)
scaler = preprocessing.StandardScaler()
kmeans= KMeans(n_clusters=1,init='k-means++',random_state=0)
for state,all_samples_in_state in enumerate(X_train_IT):
    for sample in all_samples_in_state:
        if len(clusters)==0:
            switch_state+=1
            samples_to_train.append(sample)
        else:
            sample_to_pred=scaler.transform(np.array(sample).reshape(1,-1))
            sample_to_pred=ipca.transform(sample_to_pred)[0]
            min_distance=0
            min_distance_flag=1
            final_res=-1
            for cluster_n,cluster in enumerate(clusters):
                distance=0
                for feature_n,feature in enumerate(sample_to_pred):
                    distance+=(feature-cluster[feature_n])*(feature-cluster[feature_n])
                distance=math.sqrt(distance)
                if min_distance_flag==1:
                    min_distance_flag=0
                    min_distance=distance
                    final_res=cluster_n
                elif min_distance>distance:
                    min_distance=distance
                    final_res=cluster_n
            if min_distance>max_distances[final_res]:
                final_res=-1
                switch_state+=1
                samples_to_train.append(sample)
            elif switch_state>0:
                switch_state-=1
                samples_to_train.pop(0)
        if switch_state==n_of_samples_to_train:
            temp_permanent_samples_for_train=[]
            new_permanent_samples_for_train=[]
            if len(clusters)!=0:
                for samples_in_clusters in permanent_samples_for_train:
                    new_samples=ipca.inverse_transform(samples_in_clusters)
                    new_samples=scaler.inverse_transform(new_samples)
                    temp_permanent_samples_for_train.append(new_samples)
            scaler.partial_fit(samples_to_train)
            samples_to_train=scaler.transform(samples_to_train)
            ipca.partial_fit(samples_to_train)
            samples_to_train=ipca.transform(samples_to_train)
            if len(clusters)!=0:
                for samples_in_cluster in temp_permanent_samples_for_train:
                    new_samples=scaler.transform(samples_in_cluster)
                    new_samples=ipca.transform(new_samples)
                    new_permanent_samples_for_train.append(new_samples)
            permanent_samples_for_train=new_permanent_samples_for_train
            permanent_samples_for_train.append(samples_to_train)
            kmeans=KMeans(n_clusters=len(permanent_samples_for_train),init='k-means++',random_state=0)
            kmeans_sample=[]
            for samples in permanent_samples_for_train:
                kmeans_sample.extend(samples)
            kmeans.fit(kmeans_sample)
            clusters=kmeans.cluster_centers_
            max_distances=[]
            for cluster_n,samples_in_clusters in enumerate(permanent_samples_for_train):
                max_distance=0
                distance_log=[]
                for sample in samples_in_clusters:
                    distance=0
                    for feature_n,feature in enumerate(sample):
                        distance+=(clusters[cluster_n][feature_n]-feature)*(clusters[cluster_n][feature_n]-feature)
                    distance=math.sqrt(distance)
                    distance_log.append(distance)
                distance_log.sort()
                mean=np.mean(distance_log)
                std=np.std(distance_log)
                max_distances.append(10)
            print("Detected new state:",state)
            switch_state=0
            samples_to_train=[]

            

                

corr=0
err=0
res=[]
x_axis=[]
y_axis=[]
z_axis=[]
label=[]

for state,all_samples_in_state in enumerate(X_test_IT):
    for sample in all_samples_in_state:
        min_distance=0
        min_distance_flag=1
        final_res=0
        sample_to_pred=scaler.transform(np.array(sample).reshape(1,-1))
        sample_to_pred=ipca.transform(np.array(sample_to_pred))
        x_axis.append(sample_to_pred[0][0])
        y_axis.append(sample_to_pred[0][1])
        z_axis.append(sample_to_pred[0][2])
        label.append(state)

        #check which cluster is this point closest too
        for cluster_n,cluster in enumerate(clusters):
            distance=0
            for feature_n,feature in enumerate(sample_to_pred[0]):
                distance+=(cluster[feature_n]-feature)*(cluster[feature_n]-feature)
            distance=math.sqrt(distance)
            if min_distance_flag==1:
                min_distance_flag=0
                min_distance=distance
                final_res=cluster_n
            if min_distance>distance:
                min_distance=distance
                final_res=cluster_n

        #check if the point is within the range of the closest cluster, if it isint then its either an outlier or a new state

        if min_distance>max_distances[final_res]:
            final_res=-1

        res.append(final_res)
        if final_res==state:
            corr+=1
        else:
            err+=1
data={'x_axis':x_axis,'y_axis':y_axis,'z_axis':z_axis,'labels':label}
df=pd.DataFrame(data)
fig = px.scatter_3d(df, x='x_axis', y='y_axis', z='z_axis',color='labels')
fig.update_layout(height=600, width=600,title_text="points of data")
fig.show()
print(corr)
print(err)
cm=confusion_matrix(y_test[:len(res)],res)
print("Accuracy:",corr/(corr+err)*100)
print(cm)
print(clusters)
print(max_distances)


            

IF+PCA

In [ ]:
os.chdir(init_dir)
os.chdir(r'..\Data\actual_new_motor\Features')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn import preprocessing
from isotree import IsolationForest
from sklearn.decomposition import IncrementalPCA

import sys

peaks_or_fft=0
dataset_ratio=0.7
n_of_peaks_to_use=3
use_of_amps=1
sort=1
add_energy=1
add_mean=0
add_std=1
n_of_samples_per_class=-1

all_files=os.listdir(os.getcwd())
FFTs=np.zeros((len(all_files),3),dtype=object)
peaks=np.zeros((len(all_files),3),dtype=object)
peak_amp=np.zeros((len(all_files),3),dtype=object)
mean_std=np.zeros((len(all_files),3),dtype=object)
for class_n,file in enumerate(all_files):
    feature_file=open(file,'r')
    result_data_raw=feature_file.readlines()
    FFTs[class_n],peaks[class_n],peak_amp[class_n],mean_std[class_n]=process_features(result_data_raw,n_of_samples_per_class)

X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std)

X_train_IT=[]
X_test_IT=[]
y_test_IT=[]
current_state=-1
X_train_aux=[]
test=0
for sample_n,sample in enumerate(X_train):
    if(current_state==-1):
        current_state=y_train[sample_n]
    if(y_train[sample_n]==current_state):
        X_train_aux.append(sample)
    else:
        X_train_IT.append(X_train_aux)
        X_train_aux=[]
    current_state=y_train[sample_n]
X_train_IT.append(X_train_aux)

current_state=-1
X_test_aux=[]
y_test_aux=[]
for sample_n,sample in enumerate(X_test):
    if(current_state==-1):
        current_state=y_test[sample_n]
    if(y_test[sample_n]==current_state):
        X_test_aux.append(sample)
        y_test_aux.append(y_test[sample_n])
    else:
        X_test_IT.append(X_test_aux)
        y_test_IT.append(y_test_aux)
        X_test_aux=[]
        y_test_aux=[]
    current_state=y_test[sample_n]
X_test_IT.append(X_test_aux)
y_test_IT.append(y_test_aux)


clf=[]
switch_state=0
n_of_samples_to_train=50
thresholds=[]
samples_to_train=[]
permanent_samples_for_train=[]
low_dimention=20
ipca=IncrementalPCA(n_components=low_dimention)
scaler = preprocessing.StandardScaler()

for state,samples_in_state in enumerate(X_train_IT):
    for sample in samples_in_state:
        if len(clf)==0:
            switch_state+=1
            samples_to_train.append(sample)
        else:
            new_sample=scaler.transform(np.array(sample).reshape(1,-1))
            new_sample=ipca.transform(new_sample)
            min_score=0
            min_score_flag=1
            final_res=-1
            for IF_n,IF in enumerate(clf):
                pred=IF.predict(new_sample)
                if min_score_flag==1:
                    min_score=pred
                    min_score_flag=0
                    final_res=IF_n
                elif min_score>pred:
                    min_score=pred
                    final_res=IF_n
            if min_score>thresholds[final_res]:
                final_res=-1
                switch_state+=1
                samples_to_train.append(sample)
            elif switch_state>0:
                switch_state-=1
                samples_to_train.pop(0)
            if(state==3):
                print(final_res)
                print(min_score)
        if switch_state==n_of_samples_to_train:
            if len(clf)!=0:
                temp_perm_samples=[]
                for samples_in_class in permanent_samples_for_train:
                    old_samples=ipca.inverse_transform(samples_in_class)
                    old_samples=scaler.inverse_transform(old_samples)
                    temp_perm_samples.append(old_samples)
            scaler.partial_fit(samples_to_train)
            new_samples_pred=scaler.transform(samples_to_train)
            ipca.partial_fit(new_samples_pred)
            if len(clf)!=0:
                permanent_samples_for_train=[]
                for samples_in_class in temp_perm_samples:
                    new_samples=scaler.transform(samples_in_class)
                    new_samples=ipca.transform(new_samples)
                    permanent_samples_for_train.append(new_samples)
            new_samples_pred=ipca.transform(new_samples_pred)
            permanent_samples_for_train.append(new_samples_pred)
            clf=[]
            thresholds=[]
            for class_n,samples_in_class in  enumerate(permanent_samples_for_train):
                temp_clf=IsolationForest(ndim=2, ntrees=100,missing_action="fail")
                temp_clf.fit(samples_in_class)
                predictions=temp_clf.predict(samples_in_class)
                mean=np.mean(predictions)
                std=np.std(predictions)
                thresholds.append(mean+3*std)
                clf.append(temp_clf)
            print("Made a new IF for state:",state)
            print("Threshold:",thresholds[-1])
            switch_state=0
            samples_to_train=[]
print(thresholds)




This next section will simply read all of the time series of a folder and reduce their sampling rate.

In [ ]:
source_sr=6666
desired_sr=1666



os.chdir(init_dir)
os.chdir(r'..\Data\actual_new_motor')
all_files=os.listdir(os.getcwd())
size_of_time_series=0
for file in all_files:
    if file[0]=='s':
        size_of_time_series+=1
time_series=np.zeros((size_of_time_series,3),dtype=object)
class_n=0
for file in all_files:
    first_val=0
    if file[0]=='s':
        time_series_file=open(file,'r')
        time_series_raw=time_series_file.readlines()
        single_time_series=[]
        single_time_series,med_res_x,med_res_y,med_res_z=process_raw(time_series_raw)
        for sample in single_time_series:
            for axis in range(3):
                if first_val<=2:
                    time_series[class_n][axis]=[sample[axis]]
                    first_val+=1
                else:
                    time_series[class_n][axis].append(sample[axis])
        class_n+=1
os.chdir('Resampled_Data')
for class_n,all_axis in enumerate(time_series):
    new_time_serie=[]
    for axis in all_axis:
        resampled_axis=librosa.resample(np.asfarray(axis),orig_sr=source_sr,target_sr=desired_sr)
        new_time_serie.append(resampled_axis)
    with open("state"+str(class_n)+".txt",'w') as file:
        for sample_n in range(len(new_time_serie[0])):
            string_to_write=str(new_time_serie[0][sample_n])+" "+str(new_time_serie[1][sample_n])+" "+str(new_time_serie[2][sample_n])+'\n'
            file.write(string_to_write)



This section is dedicated to showing and comparing the results from the ON/OFF gathered from the 3D printers experiment. Some results are the fact that it is almost 100% accurate when the 3D printer is not working. When it is working it seems to be somewhat inconsistent. This is due to the fact that:

1- The algorithm was not trained especificaly for the 3D printer, therefore, in this case, it is insensative to the vibration.

2- The 3D printer itself can be stationary while working which results in the algorithm detecting it as OFF while it is tecnacly working.

This experiment serves to show that while the LSM6DSOX MLC is a very useful tool to detect the ON/OFF state of a machine, it should be accompanied by some sort of software to increase accuracy. Some basic and efficient solutions could a bigger meta-classifier or increase the amount of samples utilized in the MLC. In this experiment, 10 samples were used while the accelerometer was working at 26Hz with a meta-classifier of 5 samples, which means if the machine is still(or with low vibration) for (10/26)*5=1.923s it will register as an OFF, which is common for a 3D printer.

In [2]:
import csv
import datetime
import plotly.express as px
import plotly.graph_objects as go

os.chdir(init_dir)
os.chdir(r'..\Data\3d_printer_on_off\Values')

df_flag=1
df=pd.DataFrame()
files=os.listdir('.')
numbers=[]
final_files=[]
for filename in files:
    try:
        digits=[char for char in filename if char.isdigit()]
        numbers.append(int(''.join(num for num in digits)))
        final_files.append(filename)
    except:
        print("File ",filename," does not have a number, therefor shouldent be in this folder.")
sorted_files=sorted(zip(numbers,files))
numbers,files=zip(*sorted_files)
files=list(files)
for filename in files:
    times=[]
    values=[]
    with open(filename,'r') as csvfile:
        csvreader=csv.reader(csvfile)
        for row in csvreader:
            non_ms_row=row[0].split('.')[0]
            if(non_ms_row!='ts'):
                times.append(non_ms_row)
                values.append(int(row[1]))
        sub_data={'ts':times[::-1],'ON_OFF':values[::-1]}
        if df_flag==1:
            df=pd.DataFrame(sub_data)
            df_flag=0
        else:
            sub_df=pd.DataFrame(sub_data)
            df=pd.concat([df,sub_df],ignore_index=True)

os.chdir(init_dir)
os.chdir(r'..\Data\3d_printer_on_off\Ground_Truth')
intervals=[]
with open('Ground_Truth_ON_OFF.csv','r') as csvfile:
    csvreader=csv.reader(csvfile)
    print_times=[]
    print_flag=0
    for row in csvreader:
        if print_flag==0 and row[1]=='Start':
            datetime_obj=datetime.datetime.strptime(row[0],'%m/%d/%Y %H:%M')
            print_times.append(datetime_obj)
            print_flag=1
        elif print_flag==1 and row[1]=='Stop':
            datetime_obj=datetime.datetime.strptime(row[0],'%m/%d/%Y %H:%M')
            print_times.append(datetime_obj)
            print_flag=0
            intervals.append(print_times)
            print_times=[]
        else:
            print("Something is wrong")
err=0
corr=0
ts=[]
pred=[]
GT=[]
for index,df_row in df.iterrows():
    if df_row.iloc[0]!='ts':
        pred.append(df_row.iloc[1])
        registered_time=datetime.datetime.strptime(df_row.iloc[0],'%Y-%m-%d %H:%M:%S')
        ts.append(registered_time)
        prev_time=datetime.datetime(2024,7,11,1,10,10)
        time_flag=0
        for print_time in intervals:
            if registered_time<print_time[0] and registered_time<prev_time:
                GT.append(0)
                if df_row.iloc[1]==0:
                    corr+=1
                else:
                    err+=1
                time_flag=1
                break
            if registered_time>print_time[0] and registered_time<print_time[1]:
                GT.append(1)
                if df_row.iloc[1]==1:
                    corr+=1
                else:
                    err+=1
                time_flag=1
                break
            prev_time=print_time[1]
        if time_flag==0:
            GT.append(0)
            if df_row.iloc[1]==0:
                corr+=1
            else:
                err+=1
print("Accuracy:",corr/(corr+err)*100,"%")
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts, y=pred,
                    mode='lines',
                    name='Predicted values'))
fig.add_trace(go.Scatter(x=ts, y=GT,
                    mode='lines',
                    name='Ground Truth'))
fig.show()




Accuracy: 96.91979852770244 %


These next few sections will be the next steps after kmeans on training on the esp32. The idea is now that we have the data classified we can train a supervised model and have a better end result since it can be optimized in terms of battery, memory and computational power.

So far, the workflow is:

-Gather data(both the features of the states and the log of the K-means algorithm)

-Train a model, most likely a RF since it was the most promissing ML

-Gather the log of the final classifier

-Compare and justify results.


This section is where the data is gathered from the ESP32. It is crutial that the code being run on the ESP is the "Kmeans_solo" code since this is the one that sends the features in the correct format. The features will be seperated into 3 files per classes: FFT,std and peak. This solution is not elegant or optimal, however it works therefore i will only optimize it if i have additional time. 

In [8]:
from serial import Serial
import time
import keyboard

os.chdir(init_dir)
os.chdir(r'..\Data\temp_log')
ser=Serial('COM8',baudrate=115200)
stop=time.time()+10000
log_number=0
log_lines=0
class_number=0
log_name="log"+str(log_number)+".txt"
log_file=open(log_name,'w')
FFT_files=[open("FFT{}.txt".format(i),'w') for i in range(20)]
std_files=[open("std{}.txt".format(i),'w') for i in range(20)]
peak_files=[open("peak{}.txt".format(i),'w') for i in range(20)]
peak_vals=[]
peak_amp=[]
class_counter=-1
print("Starting:")
line=""
while True:
    if keyboard.is_pressed('space'):
        break
    if ser.in_waiting!=0:
        try:
            line_binary=ser.read(ser.in_waiting)
            line+=line_binary.strip().decode('utf-8')
            if line_binary[-1]==10:
                if(line[0]=='*'):
                    sub_line=line.split(':')
                    class_n=int(sub_line[0][1])
                    log_file.write(str(class_n))
                    log_file.write("\n")
                    for idx,val in enumerate(sub_line[1].split(',')): 
                        FFT_files[class_n].write(val)
                        if idx<len(sub_line[1].split(','))-1:
                            FFT_files[class_n].write(',')
                    FFT_files[class_n].write("\n")

                    for idx,val in enumerate(sub_line[2].split(',')):
                        std_files[class_n].write(val)
                        if idx<len(sub_line[2].split(','))-1:
                            std_files[class_n].write(',')
                    std_files[class_n].write("\n")

                    for idx,val in enumerate(sub_line[3].split(',')):
                        if idx%2==0:
                            peak_vals.append(val)
                        else:
                            peak_amp.append(val)
                    for idx,val in enumerate(peak_vals):
                        peak_files[class_n].write(val)
                        if idx<len(peak_vals)-1:
                            peak_files[class_n].write(',')
                    peak_files[class_n].write("\n")
                    for idx,val in enumerate(peak_amp):
                        peak_files[class_n].write(val)
                        if idx<len(peak_amp)-1:
                            peak_files[class_n].write(',')
                    peak_files[class_n].write("\n")
                    peak_vals=[]
                    peak_amp=[]

                    print("Got sample of class:{}".format(class_n))
                    
                if(line[0]=='+'):
                    for val in line:
                        if val!='+':
                            log_file.write(val)
                    log_file.write("\n")
                    log_lines+=1
                line=""
        except: 
            print("Something went wrong, please make sure the code on the ESP is the Kmeans_solo.")
log_file.close()
for idx in range(len(FFT_files)):
    file_name=FFT_files[idx].name
    FFT_files[idx].seek(0,os.SEEK_END)
    if FFT_files[idx].tell()==0:
        FFT_files[idx].close()
        os.remove(file_name)
    else:
        FFT_files[idx].close()
for idx in range(len(std_files)):
    file_name=std_files[idx].name
    std_files[idx].seek(0,os.SEEK_END)
    if std_files[idx].tell()==0:
        std_files[idx].close()
        os.remove(file_name)
    else:
        std_files[idx].close()
for idx in range(len(peak_files)):
    file_name=peak_files[idx].name
    peak_files[idx].seek(0,os.SEEK_END)
    if peak_files[idx].tell()==0:
        peak_files[idx].close()
        os.remove(file_name)
    else:
        peak_files[idx].close()
ser.close()

Starting:
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0
Got sample of class:0


This section is where the final classifier will be built. As it stands, it only trains the RF model.

In [2]:
os.chdir(init_dir)
os.chdir(r'..\Data\cell_phone')
from sklearn.metrics import confusion_matrix
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from micromlgen import port

all_files=os.listdir(os.getcwd())
n_of_classes=0
for file in all_files:
    if file[0]=='F':
        n_of_classes+=1
FFTs=np.zeros((n_of_classes,3),dtype=object)
peaks=np.zeros((n_of_classes,3),dtype=object)
peak_amp=np.zeros((n_of_classes,3),dtype=object)
mean_std=np.zeros((n_of_classes,3),dtype=object)

peaks_or_fft=1
dataset_ratio=0.33
n_of_peaks_to_use=6
use_of_amps=1
sort=1
add_energy=0
add_mean=0
add_std=0
n_of_samples_per_class=-1

for file_n,file in enumerate(all_files):
    numbers = [i for i in file.split()[0] if i.isdigit()]
    class_n=""
    for number in numbers:
        class_n+=number
    if(class_n!=""):
        class_n=int(class_n)
        #get FFTs
        if file[0]=='F':
            class_n=int(file[3])
            feature_file=open(file,'r')
            result_data_raw=feature_file.readlines()
            for idx,line in enumerate(result_data_raw):
                str_list=list(line.strip().split(","))
                float_list=[]
                for val in str_list:
                    if val!='':
                        float_list.append(float(val))
                for axis in range(3):
                    aux=[]
                    if idx==0:
                        FFTs[class_n][axis]=[]
                    for n_of_FFT in range(256):
                        aux.append(float_list[n_of_FFT+axis*256])
                    FFTs[class_n][axis].append(aux)
            feature_file.close()
        #get std (im not getting mean because it tends to be useless)
        if file[0]=='s' and add_std:
            feature_file=open(file,'r')
            result_data_raw=feature_file.readlines()
            for idx,line in enumerate(result_data_raw):
                str_list=list(line.strip().split(","))
                float_list=[]
                for val in str_list:
                    if val!='':
                        float_list.append(float(val))
                for axis in range(3):
                    aux=[]
                    if idx==0:
                        mean_std[class_n][axis]=[]
                    aux.append([0,float_list[axis]])
                    mean_std[class_n][axis].append(aux)
            feature_file.close()

        #get peaks
        if file[0]=='p':
            feature_file=open(file,'r')
            result_data_raw=feature_file.readlines()
            for idx,line in enumerate(result_data_raw):
                str_list=list(line.strip().split(","))
                float_list=[]
                for val in str_list:
                    if val!='':
                        float_list.append(float(val))
                for axis in range(3):
                    aux=[]
                    if idx==0:
                        peaks[class_n][axis]=[]
                        peak_amp[class_n][axis]=[]
                    for n_peaks in range(6):
                        aux.append(float_list[n_peaks+6*axis])
                    if idx%2==0:
                        peaks[class_n][axis].append(aux)
                    else:
                        peak_amp[class_n][axis].append(aux)
            feature_file.close()

X_train,y_train,X_test,y_test=build_dataset(FFTs,peaks,peak_amp,peaks_or_fft,mean_std,dataset_ratio,use_of_amps,n_of_peaks_to_use,sort,add_energy,add_mean,add_std)
print(X_train[0])
clf=RandomForestClassifier(n_estimators=5)
clf.fit(X_train,y_train)
test_labels=clf.predict(X_test)
cm=confusion_matrix(y_test,test_labels)
print(clf.score(X_test,y_test))
print(cm)
os.chdir('Models')
with open("RF_model.h",'w') as file:
    file.write(port(clf))
    #write some variables for the final classifier
    file.write("\n")
    file.write("#define USE_PEAK {} \n".format(peaks_or_fft))
    file.write("#define PEAK_N {} \n".format(n_of_peaks_to_use))
    file.write("#define USE_STD {} \n".format(add_std))
    file.write("#define SORT {} \n".format(sort))
    file.close()

[-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 729.09, 13.04, 859.29, 14.64, 1015.52, 12.47, 1119.68, 15.16, 1327.99, 12.82, 1510.27, 13.44, 104.16, 85.12, 338.51, 85.5, 885.33, 86.12, 1458.19, 95.5, 2083.13, 85.12, 3150.73, 95.5]
1.0
[[66  0  0]
 [ 0 66  0]
 [ 0  0 66]]


This section is dedicated to logging the final classifier.

In [3]:
from serial import Serial
import keyboard

os.chdir(init_dir)
os.chdir(r'..\Data\temp_log')
ser=Serial('COM8',baudrate=115200)
log_number=0
log_lines=0
class_number=0
log_name="RF_log"+str(log_number)+".txt"
log_file=open(log_name,'w')
class_counter=-1
print("Logging:")
line=""
while True:
    if keyboard.is_pressed('space'):
        break
    if ser.in_waiting!=0:
        try:
            line_binary=ser.read(ser.in_waiting)
            line+=line_binary.strip().decode('utf-8')
            if line_binary[-1]==10: 
                sub_line=line.split(':')   
                if(sub_line[2].isnumeric()):
                    for val in sub_line[2]:
                        log_file.write(val)
                    log_file.write("\n")
                    log_lines+=1
                line=""
        except: 
            print("Something went wrong, please make sure the code on the ESP32 is the Final_classifier.")
log_file.close()
ser.close()

Starting:


Finally, some plots and results from the logs

In [13]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

os.chdir(init_dir)
os.chdir(r'..\Data\temp_log')

kmeans_file=open('log0.txt','r')
RF_file=open('RF_log0.txt','r')
kmeans_res=[]
RF_res=[]
raw_data=kmeans_file.readlines()
for line in raw_data:
    kmeans_res.append(int(line))
raw_data=RF_file.readlines()
for line in raw_data:
    RF_res.append(int(line))
fig = make_subplots(rows=2, cols=1,subplot_titles=("Unsupervised results(Kmeans)", "Supervised results(Random Forest)"))
x=[i for i in range(len(kmeans_res))]
fig.add_trace(
    go.Scatter(x=[i for i in range(len(kmeans_res))], y=kmeans_res,name="K-means"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=[i for i in range(len(RF_res))], y=RF_res,name="Random Forest"),
    row=2, col=1
)
fig.update_layout(height=600, width=1200, title_text="Predictions of both logs",)
fig.show()